In [ ]:
!nvidia-smi

Wed Apr 20 11:40:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 10.0 MB/s 
     |████████████████████████████████| 895 kB 61.7 MB/s 
     |████████████████████████████████| 596 kB 62.7 MB/s 
     |████████████████████████████████| 77 kB 7.1 MB/s 
     |████████████████████████████████| 6.6 MB 53.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Read the following link to get kaggle.json https://www.kaggle.com/general/74235


In [ ]:
%%shell
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c fake-news

 77% 36.0M/46.5M [00:00<00:00, 164MB/s]
100% 46.5M/46.5M [00:00<00:00, 168MB/s]


In [ ]:
!unzip /content/fake-news.zip

Archive:  /content/fake-news.zip
  inflating: submit.csv              
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
from tqdm.auto import tqdm
import pandas as pd
import tensorflow as tf
import numpy as np
import string
import re
from os.path import join
import os
from transformers import AutoTokenizer,TFAutoModelForSequenceClassification,AutoModelForSequenceClassification,create_optimizer,get_scheduler,get_polynomial_decay_schedule_with_warmup
from transformers import AutoConfig
from itertools import chain
from collections import Counter
from torch.utils.data import DataLoader
import torch
from torch import nn
from torch.utils.data import Dataset
import nltk
from nltk.corpus import stopwords
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
fake_news_folder="fake_news"
fake_new_dataset="raw_dataset"
fake_new_models="models"
name="bert-base-uncased"
framework='pytorch'

In [ ]:
root_path="/content/drive/MyDrive"
fake_news_folder_path=join(root_path,fake_news_folder)
fake_news_dataset_path=join(root_path,fake_news_folder,fake_new_dataset)
fake_news_models_path=join(root_path,fake_news_folder,fake_new_models)
model_path=f'{fake_news_models_path}/{name}/'
framework_path=f'{model_path}{framework}/'

In [ ]:
os.makedirs(model_path, exist_ok=True)
os.makedirs(fake_news_dataset_path, exist_ok=True)
os.makedirs(framework_path, exist_ok=True)

# Load fake news dataset from kaggle

In [ ]:
df=pd.read_csv("/content/train.csv",index_col=False)
df=df.fillna("")
df

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1
...,...,...,...,...,...
20795,20795,Rapper T.I.: Trump a ’Poster Child For White S...,Jerome Hudson,Rapper T. I. unloaded on black celebrities who...,0
20796,20796,"N.F.L. Playoffs: Schedule, Matchups and Odds -...",Benjamin Hoffman,When the Green Bay Packers lost to the Washing...,0
20797,20797,Macy’s Is Said to Receive Takeover Approach by...,Michael J. de la Merced and Rachel Abrams,The Macy’s of today grew from the union of sev...,0
20798,20798,"NATO, Russia To Hold Parallel Exercises In Bal...",Alex Ansary,"NATO, Russia To Hold Parallel Exercises In Bal...",1


In [ ]:
def merge(title,text):
  merged=[]
  for i in zip(title,text):
    if i[0].lower() not in i[1].lower():
      merged.append(i[0]+" "+i[1])
    else:
      merged.append(i[1])
  return merged

In [ ]:
merged=merge(df['title'].tolist(),df['text'].tolist())

In [ ]:
df['merged']=merged

In [ ]:
df=df.drop(['id','author'],axis=1)
df=df.sample(frac=1)
df

,title,text,label,merged
6871,Harry Reid Vindicated: The FBI Is Sitting On I...,Harry Reid Vindicated: The FBI Is Sitting On I...,1,Harry Reid Vindicated: The FBI Is Sitting On I...
18517,Muslims Are Kidnapping White Girls And Forcing...,Print \nIslam and sex slavery are like peanut ...,1,Muslims Are Kidnapping White Girls And Forcing...
7070,Studie beweist: Indianer kennen doch Schmerz,"Donnerstag, 27. Oktober 2016 Studie beweist: I...",1,"Donnerstag, 27. Oktober 2016 Studie beweist: I..."
5351,The Other Trump - The New York Times,Like place cards at a society wedding or the l...,0,The Other Trump - The New York Times Like plac...
9402,Israeli troops demolish three Palestinian home...,Palestine Palestinians inspect the damaged hou...,1,Israeli troops demolish three Palestinian home...
...,...,...,...,...
9125,"Hillary Insisted America Fact Check Her, So We...",Home / News / Hillary Insisted America Fact Ch...,1,Home / News / Hillary Insisted America Fact Ch...
17570,NYPD: Hillary Clinton ‘Child Sex Scandal’ Abou...,NYPD sources said these new emails include evi...,1,NYPD: Hillary Clinton ‘Child Sex Scandal’ Abou...
16442,"Israel settlements legal, Trump aide says, pla...","Israel settlements legal, Trump aide says, pla...",1,"Israel settlements legal, Trump aide says, pla..."
4662,4 Die in Jerusalem Attack as Palestinian Rams ...,JERUSALEM — A Palestinian driver plowed a t...,0,4 Die in Jerusalem Attack as Palestinian Rams ...


Split raw dataset to train set and validation set

In [ ]:
train=df.iloc[:-2600]
valid=df.iloc[-2600:]

In [ ]:
len(train),len(valid)

(18200, 2600)

Check data whether it is balanced or not

In [ ]:
for i in range(2):
  samples=train[train['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 9099 samples
label: 1	has 9101 samples


In [ ]:
for i in range(2):
  samples=valid[valid['label']==i]
  print(f'label: {i}\thas {len(samples)} samples')

label: 0	has 1288 samples
label: 1	has 1312 samples


In [ ]:
train.to_csv(join(fake_news_dataset_path,"train.csv"),index=False)

In [ ]:
valid.to_csv(join(fake_news_dataset_path,"valid.csv"),index=False)

# Clean and preprocessing

##Basic clean data

In [ ]:
train=pd.read_csv(join(fake_news_dataset_path,"train.csv"),index_col=False)
train=train.dropna(subset=['label', 'merged'])

valid=pd.read_csv(join(fake_news_dataset_path,"valid.csv"),index_col=False)
valid=valid.dropna(subset=['label', 'merged'])


In [ ]:
contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}

In [ ]:
list_stopwords=stopwords.words('english')

In [ ]:
def white_space_fix(text):
  return ' '.join(text.split())

In [ ]:
def normalize(s):
  def lower(text):
    return text.lower()
  
  def norm_apostrophe(text):
    return re.sub("’(\s|)","'",text)

  def replace_contraction(text):
    for k,v in contraction_mapping.items():
      text=text.replace(k,v)
    return text
  def remove_punc(text):
    return re.sub("\W"," ",text)

  def remove_articles(text):
    regex = re.compile(r'\b(a|an|the)\b', re.UNICODE)
    return re.sub(regex, ' ', text)

  def norm_number(text):
    new_text=text
    pattern = re.compile(r'\b((\d+)[.,\s]+(\d+))\b',re.UNICODE)

    founds=re.findall(pattern,new_text)
    for i in founds:
      new_text=re.sub(i[0],"".join(i[1:]),new_text)
    return new_text

  def remove_stopwords(text):
    new_text=text
    pattern=r'\b({})\b'.format(f'({"|".join(list_stopwords)})')
    regex = re.compile(pattern, re.UNICODE)
    new_text=re.sub(regex, ' ', new_text)
    return new_text

  def remove_nickname_twitter(text):
    pattern=r'\(@\b(.+)\b\)'
    return re.sub(pattern," ",text)

  def remove_link(text):
    pattern=r'\b((?:https?://)?(?:(?:www\.)?(?:[\da-z\.-]+)\.(?:[a-z]{2,6})|(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)|(?:(?:[0-9a-fA-F]{1,4}:){7,7}[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,7}:|(?:[0-9a-fA-F]{1,4}:){1,6}:[0-9a-fA-F]{1,4}|(?:[0-9a-fA-F]{1,4}:){1,5}(?::[0-9a-fA-F]{1,4}){1,2}|(?:[0-9a-fA-F]{1,4}:){1,4}(?::[0-9a-fA-F]{1,4}){1,3}|(?:[0-9a-fA-F]{1,4}:){1,3}(?::[0-9a-fA-F]{1,4}){1,4}|(?:[0-9a-fA-F]{1,4}:){1,2}(?::[0-9a-fA-F]{1,4}){1,5}|[0-9a-fA-F]{1,4}:(?:(?::[0-9a-fA-F]{1,4}){1,6})|:(?:(?::[0-9a-fA-F]{1,4}){1,7}|:)|fe80:(?::[0-9a-fA-F]{0,4}){0,4}%[0-9a-zA-Z]{1,}|::(?:ffff(?::0{1,4}){0,1}:){0,1}(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])|(?:[0-9a-fA-F]{1,4}:){1,4}:(?:(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])\.){3,3}(?:25[0-5]|(?:2[0-4]|1{0,1}[0-9]){0,1}[0-9])))(?::[0-9]{1,4}|[1-5][0-9]{4}|6[0-4][0-9]{3}|65[0-4][0-9]{2}|655[0-2][0-9]|6553[0-5])?(?:/[\w\.-]*)*/?)\b'
    return re.sub(pattern," ",text)

  def remove_single_character(text):
    return re.sub(r'\b([a-zA-Z])\b'," ",text)

  new_s=remove_link(s)
  new_s=remove_nickname_twitter(new_s)
  new_s=lower(new_s)
  new_s=norm_apostrophe(new_s)
  new_s=replace_contraction(new_s)
  new_s=norm_number(new_s)
  new_s=remove_punc(new_s)
  new_s=remove_articles(new_s)
  new_s=remove_single_character(new_s)
  new_s=remove_stopwords(new_s)

  return white_space_fix(new_s).strip()


In [ ]:
train['merged']=train['merged'].map(normalize)

In [ ]:
def find_top_common_tokens(list_texts,top_n=30):
  tokens=list(chain.from_iterable([i.split() for i in list_texts]))
  counter=Counter(tokens)
  top_common_tokens=dict(counter.most_common(top_n))
  return list(top_common_tokens.keys())

In [ ]:
top_common_tokens=find_top_common_tokens(train['merged'].tolist())

In [ ]:
def remove_top_common_tokens(text):

  pattern=r'\b({})\b'.format(f'({"|".join(top_common_tokens)})')
  regex = re.compile(pattern, re.UNICODE)
  text=re.sub(regex, ' ', text)
  return white_space_fix(text).strip()

In [ ]:
train['merged']=train['merged'].map(remove_top_common_tokens)

In [ ]:
valid['merged']=valid['merged'].map(normalize)

In [ ]:
train=train[train['merged']!=""]
valid=valid[valid['merged']!=""]

In [ ]:
len(train),len(valid)

(18198, 2600)

In [ ]:
train.to_csv(join(model_path,"train.csv"),index=False)
valid.to_csv(join(model_path,"valid.csv"),index=False)

##Preprocessing

In [ ]:
train=pd.read_csv(join(model_path,"train.csv"),index_col=False)
valid=pd.read_csv(join(model_path,"valid.csv"),index_col=False)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(name)

In [ ]:
max_length=512

In [ ]:
train_batch_size=8
num_training_samples=len(train)
num_training_batches=int((num_training_samples/train_batch_size)+0.5)

In [ ]:
valid_batch_size=8
num_valid_samples=len(valid)
num_valid_batches=int((num_valid_samples/valid_batch_size)+0.5)

## Pytorch

In [ ]:
class FakeNewsDataset(Dataset):
    def __init__(self, input_ids, attention_mask,token_type_ids,labels,device):
        self.labels = labels
        self.input_ids=input_ids
        self.attention_mask=attention_mask
        self.device=device
        
    def __len__(self):
            return len(self.labels)
    def __getitem__(self, idx):
        label = self.labels[idx]
        input_ids = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        sample = {"input_ids": input_ids.to(device),'attention_mask':attention_mask.to(device), "labels": label.to(device)}
        return sample

In [ ]:
return_tensors='pt'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

Train set

In [ ]:
x_train=tokenizer(train['merged'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [ ]:
y_train=torch.Tensor(train['label']).to(torch.long)

In [ ]:
train_dataset={i:x_train[i] for i in x_train}
train_dataset.update({"labels":y_train})

In [ ]:
train_dataset=FakeNewsDataset(**train_dataset,device=device)

In [ ]:
train_dataset=DataLoader(train_dataset,batch_size=train_batch_size,drop_last=True)

Valid set

In [ ]:
x_valid=tokenizer(valid['merged'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)#.to(device)

In [ ]:
y_valid=torch.Tensor(valid['label']).to(torch.long)#.to(device)

In [ ]:
valid_dataset={i:x_valid[i] for i in x_valid}
valid_dataset.update({"labels":y_valid})

In [ ]:
valid_dataset=FakeNewsDataset(**valid_dataset,device=device)

In [ ]:
valid_dataset=DataLoader(valid_dataset,batch_size=valid_batch_size,drop_last=True)

## Tensorflow

Train set

In [ ]:
x_train=tokenizer(train['merged'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [ ]:
y_train=tf.convert_to_tensor(train['label'],dtype=tf.int32)

In [ ]:
train_dataset={i:x_train[i] for i in x_train}
train_dataset.update({"labels":y_train})

In [ ]:
train_dataset=tf.data.Dataset.from_tensor_slices(train_dataset)

In [ ]:
train_dataset=train_dataset.batch(train_batch_size, drop_remainder=True)

Valid set

In [ ]:
x_valid=tokenizer(valid['text'].tolist() ,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors)

In [ ]:
y_valid=tf.convert_to_tensor(valid['label'],dtype=tf.int32)

In [ ]:
valid_dataset={i:x_valid[i] for i in x_valid}
valid_dataset.update({"labels":y_valid})
valid_dataset=tf.data.Dataset.from_tensor_slices(valid_dataset)

In [ ]:
valid_dataset=valid_dataset.batch(valid_batch_size, drop_remainder=True)

# Train

In [ ]:
epochs=5
num_training_steps=epochs * len(train_dataset)
num_warmup_steps=800
init_lr=1e-5
min_lr_ratio=1e-2

In [ ]:
num_training_steps,num_warmup_steps

(11370, 800)

## Pytorch

Load the pretrained model

In [ ]:
num_labels=len(train['label'].unique())
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels
model = AutoModelForSequenceClassification.from_pretrained(name,config=config).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

- optimizer -> adam
- lr_schdeluer -> polynomial

In [ ]:
# checkpoint = torch.load("/content/drive/MyDrive/fake_news/models/bert-base-uncased/pytorch/ckpt4.pt")
# model.load_state_dict(checkpoint['model_state_dict'])

In [ ]:
optimizer=torch.optim.Adam(model.parameters(),lr=init_lr)
lr_scheduler=get_polynomial_decay_schedule_with_warmup(optimizer,num_warmup_steps,num_training_steps,lr_end =init_lr*min_lr_ratio)
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
def save_model(epoch,model,optimizer,loss,path):
  torch.save({
          'epoch': epoch,
          'model_state_dict': model.state_dict(),
          'optimizer_state_dict': optimizer.state_dict(),
          'loss': loss,
          }, join(path,f'ckpt{epoch}.pt'))

In [ ]:
def run_train(optimizer,dataloader,lr_scheduler):
  model.train(True)
  loop = tqdm(dataloader)
  
  loss=0
  for batch in dataloader:
    # model.zero_grad(set_to_none=True)
    optimizer.zero_grad()
    outputs=model(**batch)

    outputs.loss.backward()
    _loss=outputs.loss

    loss+=_loss.item()
    optimizer.step()
    lr_scheduler.step()
    loop.set_postfix({f'train_loss_per_batch':_loss.item()})
    loop.update()
    
  return loss/len(dataloader)

In [ ]:
def run_valid(dataloader):
  loop = tqdm(dataloader)
  model.eval()
  loss=0
  with torch.no_grad():
    for batch in dataloader:
      outputs=model(**batch)
      _loss=outputs.loss

      loss+=_loss.item()
      loop.set_postfix({f'valid_loss_per_batch':_loss.item()})
      loop.update()
  return loss/len(dataloader)

In [ ]:
for epoch in range(epochs):
  train_loss=run_train(optimizer,train_dataset,lr_scheduler)
  valid_loss=run_valid(valid_dataset)
  print(f'epoch: {epoch} train_loss: {train_loss} valid_loss: {valid_loss}')
  save_model(epoch,model,optimizer,valid_loss,framework_path)

  0%|          | 0/2274 [00:00<?, ?it/s]

## Tensorflow

In [ ]:
num_labels=len(train['label'].unique())
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels
model = TFAutoModelForSequenceClassification.from_pretrained(name,config=config)

In [ ]:
loss_fn=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction=tf.keras.losses.Reduction.AUTO)

- optimizer -> adam
- lr_schdeluer -> polynomial

In [ ]:
optimizer,_=create_optimizer(init_lr=init_lr,num_train_steps =num_training_steps ,num_warmup_steps=num_warmup_steps,adam_epsilon =1e-8,min_lr_ratio =min_lr_ratio )

Define train_step function for training

In [ ]:
train_step_signature= [
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, None), dtype=tf.int32),
    tf.TensorSpec(shape=(None, ), dtype=tf.int32),
]


@tf.function(input_signature=train_step_signature)
def train_step(input_ids, attention_mask,labels):
  

  with tf.GradientTape() as tape:

    outputs= model(input_ids=input_ids,attention_mask=attention_mask,training = True)
    loss=loss_fn(labels,outputs.logits)

  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return loss

Defind test_step function for validation

In [ ]:
ckpt_path = framework_path
max_to_keep=5

ckpt = tf.train.Checkpoint(model=model,
                           optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, ckpt_path, max_to_keep=max_to_keep)

# if a checkpoint exists, restore the latest checkpoint.
if ckpt_manager.latest_checkpoint:
  ckpt.restore(ckpt_manager.latest_checkpoint)
  print(f'Loading the latest checkpoint from {ckpt_manager.latest_checkpoint}')
else:
  print('Initializing the new checkpoint')

In [ ]:
def test_step(input_ids, attention_mask,labels):
  
  outputs= model(input_ids=input_ids,attention_mask=attention_mask,training = False)
  loss=loss_fn(labels,outputs.logits)

  return loss

In [ ]:
def run_step(dataset,session='train'):
  
  step=train_step if session=='train' else test_step

  loop = tqdm(dataset)
  loss=0

  num_batches=0
  for batch in dataset:

    input_ids=batch['input_ids']
    attention_mask=batch['attention_mask']
    labels=batch['labels']

    _loss=step(input_ids=input_ids,attention_mask=attention_mask,labels=labels)


    _loss=_loss.numpy()
    loss+=_loss
    num_batches+=1
    loop.set_postfix({f'{session}_loss_per_batch':_loss})
    loop.update()


  loss=loss/num_batches
  return loss

In [ ]:
min_loss=np.inf
for epoch in range(epochs):

  """TRAINING"""
  train_loss=run_step(train_dataset)

  """Validation"""
  valid_loss=run_step(valid_dataset,session='valid')
  
  print(f'epoch: {epoch} train_loss: {train_loss} valid_loss: {valid_loss}')
  
  ckpt_save_path = ckpt_manager.save()
  print(f'Saving checkpoint for epoch {epoch} at {ckpt_save_path}')


#Load test set

In [ ]:
test=pd.read_csv("/content/test.csv")
test=test.fillna("")
test

,id,title,author,text
0,20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
1,20801,Russian warships ready to strike terrorists ne...,,Russian warships ready to strike terrorists ne...
2,20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
3,20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
4,20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...
...,...,...,...,...
5195,25995,The Bangladeshi Traffic Jam That Never Ends - ...,Jody Rosen,Of all the dysfunctions that plague the world’...
5196,25996,John Kasich Signs One Abortion Bill in Ohio bu...,Sheryl Gay Stolberg,WASHINGTON — Gov. John Kasich of Ohio on Tu...
5197,25997,"California Today: What, Exactly, Is in Your Su...",Mike McPhate,Good morning. (Want to get California Today by...
5198,25998,300 US Marines To Be Deployed To Russian Borde...,,« Previous - Next » 300 US Marines To Be Deplo...


In [ ]:
merged=merge(test['title'].tolist(),test['text'].tolist())

In [ ]:
submit=pd.read_csv("/content/submit.csv")
labels=torch.Tensor(submit['label'])

In [ ]:
test['merged']=merged

In [ ]:
test['merged']=test['merged'].astype(str).map(normalize)

In [ ]:
test=test[test['merged']!=""]

In [ ]:
samples=test['merged'].tolist()

In [ ]:
num_labels=len(submit['label'].unique())
num_labels

2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(name)

Load model

# Pytorch

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained(name)
config.num_labels=num_labels

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model=AutoModelForSequenceClassification.from_pretrained(name,config=config).to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
model.num_labels

2

In [ ]:
return_tensors='pt'

Load the weights

In [ ]:
checkpoint = torch.load("/content/drive/MyDrive/fake_news/models/bert-base-uncased/pytorch/ckpt3.pt")
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [ ]:
max_length=512

In [ ]:
model.eval()
y_pred=[]
with torch.no_grad():
  loop = tqdm(range(len(samples)))
  for idx,sample in enumerate(samples):
    inputs=tokenizer(sample,padding=True,truncation=True,max_length=max_length,return_tensors=return_tensors).to(device)
    outputs=model(**inputs)
    prob=torch.nn.functional.softmax(outputs.logits,dim=1)
    preds=torch.argmax(prob,dim=1)
    y_pred.append(preds)
    loop.update()


  0%|          | 0/5200 [00:00<?, ?it/s]

In [ ]:
y_pred=torch.Tensor(y_pred)

In [ ]:
y_pred

tensor([0., 1., 1.,  ..., 0., 1., 0.])

In [ ]:
labels

tensor([0., 1., 0.,  ..., 0., 1., 0.])

In [ ]:
print(classification_report(labels,y_pred,digits=5))

              precision    recall  f1-score   support

         0.0    0.57691   0.65584   0.61385      2339
         1.0    0.68320   0.60678   0.64272      2861

    accuracy                        0.62885      5200
   macro avg    0.63005   0.63131   0.62829      5200
weighted avg    0.63539   0.62885   0.62973      5200



In [ ]:
confusion_matrix(labels,y_pred)

array([[1534,  805],
       [1125, 1736]])

In [ ]:
torch.where(y_pred!=labels)

(tensor([   2,    3,    6,  ..., 5191, 5194, 5196]),)